<a href="https://colab.research.google.com/github/HelenNunez/BALG_Project3/blob/master/Training_Clustering_Test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
pip install --upgrade tensorflow-datasets

     |████████████████████████████████| 3.4MB 2.7MB/s 
  Found existing installation: tensorflow-datasets 2.1.0
    Uninstalling tensorflow-datasets-2.1.0:
      Successfully uninstalled tensorflow-datasets-2.1.0


In [ ]:
pip install -q tensorflow>=2 tfds-nightly matplotlib

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds 
import pandas as pd

In [2]:
tfds.list_builders()

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'ai2_arc',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'emnist',
 'eraser_multi_rc',
 'esnli',
 'eurosat',
 'fashion_mnist

In [17]:
# Loading the wikipedia dataset.
DATASET_NAME = 'wikipedia/20200301.en'
# DATASET_NAME = 'wikipedia/20190301.uk'

dataset, dataset_info = tfds.load(
    name=DATASET_NAME,
    data_dir='tmp',
    with_info=True,
    split=tfds.Split.TRAIN,
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset wikipedia downloaded and prepared to tmp/wikipedia/20200301.en/1.0.0. Subsequent calls will reuse this data.


In [18]:
print(dataset_info)

tfds.core.DatasetInfo(
    name='wikipedia',
    version=1.0.0,
    description='Wikipedia dataset containing cleaned articles of all languages.
The datasets are built from the Wikipedia dump
(https://dumps.wikimedia.org/) with one split per language. Each example
contains the content of one full Wikipedia article with cleaning to strip
markdown and unwanted sections (references, etc.).',
    homepage='https://dumps.wikimedia.org',
    features=FeaturesDict({
        'text': Text(shape=(), dtype=tf.string),
        'title': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=6033151,
    splits={
        'train': 6033151,
    },
    supervised_keys=None,
    citation="""@ONLINE {wikidump,
        author = "Wikimedia Foundation",
        title  = "Wikimedia Downloads",
        url    = "https://dumps.wikimedia.org"
    }""",
    redistribution_info=license: "This work is licensed under the Creative Commons Attribution-ShareAlike 3.0 Unported License. To view a copy of this l

In [19]:
print(dataset)

<PrefetchDataset shapes: {text: (), title: ()}, types: {text: tf.string, title: tf.string}>


In [20]:
TRAIN_NUM_EXAMPLES = dataset_info.splits['train'].num_examples
print('Total number of articles: ', TRAIN_NUM_EXAMPLES)

Total number of articles:  6033151


In [21]:
print('First article','\n======\n')
for example in dataset.take(1):
    print('Title:','\n------')
    print(example['title'].numpy().decode('utf-8'))
    print()

    print('Text:', '\n------')
    print(example['text'].numpy().decode('utf-8'))

First article 

Title: 
------
Joseph Greenberg

Text: 
------
Joseph Harold Greenberg (May 28, 1915 – May 7, 2001) was an American linguist, known mainly for his work concerning linguistic typology and the genetic classification of languages.

Life

Early life and education 

Joseph Greenberg was born on May 28, 1915 to Jewish parents in Brooklyn, New York. His first great interest was music. At the age of 14, he gave a piano concert in Steinway Hall. He continued to play the piano frequently throughout his life.

After finishing high school, he decided to pursue a scholarly career rather than a musical one. He enrolled at Columbia University in New York. During his senior year, he attended a class taught by Franz Boas concerning American Indian languages. With references from Boas and Ruth Benedict, he was accepted as a graduate student by Melville J. Herskovits at Northwestern University in Chicago. During the course of his graduate studies, Greenberg did fieldwork among the Hausa p

In [22]:
# The maximum length sentence we want for a single input in characters.
sequence_length = 200

In [33]:
def article_to_text(text):
    return np.array([char for char in text.numpy().decode('utf-8')])

# Converting each dataset item to a string ('text') instead of a dictionary ({'text', 'title'}).
dataset_text = dataset.map(
    lambda article: tf.py_function(func=article_to_text, inp=[article['text']], Tout=tf.string)
)

for text in dataset_text.take(2):
    print(text.numpy())
    print('\n')

[b'J' b'o' b's' ... b'c' b'a' b's']


[b'P' b'a' b'u' ... b'e' b'r' b's']




In [13]:
# ds = ds.take(1)  # Only take a single example
# for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
#   print(list(example.keys()))
#   text = example["text"]
#   title = example["title"]
#   print(text)
#   print(title)

['text', 'title']
tf.Tensor(b'Wikipedia b\xc3\xb6dero k\xc5\x93\xc3\xa1th amliethog e rhy\xce\xb4 g\xc4\x93 gell t\xc5\x8d un gel\xc5\x93ga. Denn gr\xc3\xaba\xc3\xb0 zero \xc5\x93\xc3\xa1thws ar \xc3\xb0echre di zera gentannwm hon.\n\n \nCategory:Articles\n\naf:Wikipedia\nals:Wikipedia\nam:\xe1\x8b\x8d\xe1\x8a\xad\xe1\x8d\x94\xe1\x8b\xb2\xe1\x8b\xab\nan:Biquipedia\nang:Wicip\xc7\xa3dia\nar:\xd9\x88\xd9\x8a\xd9\x83\xd9\x8a\xd8\xa8\xd9\x8a\xd8\xaf\xd9\x8a\xd8\xa7\narc:\xdc\x98\xdc\x9d\xdc\x9f\xdc\x9d\xdc\xa6\xdc\x95\xdc\x9d\xdc\x90\nast:Uiquipedia\nay:Wikipidiya\naz:Vikipediya\nba:\xd0\x92\xd0\xb8\xd0\xba\xd0\xb8\xd0\xbf\xd0\xb5\xd0\xb4\xd0\xb8\xd1\x8f\nbar:Wikipedia\nbat-smg:Vikiped\xc4\x97j\xc4\x97\nbe:\xd0\x92\xd1\x96\xd0\xba\xd1\x96\xd0\xbf\xd0\xb5\xd0\xb4\xd1\x8b\xd1\x8f\nbe-x-old:\xd0\x92\xd1\x96\xd0\xba\xd1\x96\xd0\xbf\xd1\x8d\xd0\xb4\xd1\x8b\xd1\x8f\nbg:\xd0\xa3\xd0\xb8\xd0\xba\xd0\xb8\xd0\xbf\xd0\xb5\xd0\xb4\xd0\xb8\xd1\x8f\nbm:Wikipedia\nbn:\xe0\xa6\x89\xe0\xa6\x87\xe0\xa6\x95\

In [8]:
# ds, info = tfds.load('wikipedia', split='train', with_info=True)
# tfds.as_dataframe(ds.take(1),info)

AttributeError: ignored

In [ ]:
#  # Run this and follow the instructions:
# from google.colab import drive
# drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Once mounted:
# You may need to change the paths to the correct one on your drive
# !cp /content/drive/My\ Drive/Project3 -r /content/

In [ ]:
# # Install Java, Spark, and Findspark
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
# !tar xf spark-2.4.6-bin-hadoop2.7.tgz
# !pip install -q findspark

# # Set Environment Variables
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# # Start a SparkSession
# import findspark
# findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (121 kB/s)
Reading package lists... Done


In [ ]:
# # Start a SparkSession
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [ ]:
# # Note this is an RDD approach, not dataframe. API is slightly different for dataframe
# from pyspark.mllib.feature import Word2Vec
# inp = spark.sparkContext.textFile("Project3/yelp-100k.txt").map(lambda row: row.split(" "))
# word2vec = Word2Vec()

In [ ]:
# # Train the model
# model = word2vec.fit(inp)

In [ ]:
# synonyms = model.findSynonyms('restaurant', 5)

# for word, cosine_distance in synonyms:
#     print("{}: {}".format(word, cosine_distance))

location: 0.7936934232711792
store: 0.7661047577857971
gym: 0.7582272291183472
hotel: 0.7312384843826294
establishment: 0.7134353518486023


In [ ]:
# def cosine_similarity(x, y):
#     return x.dot(y)/(x.norm(2)*y.norm(2))

In [ ]:
# x = model.transform('location')
# y = model.transform('restaurant')
# cosine_similarity(x, y)

0.7936935055394885

In [ ]:
# synonyms = model.findSynonyms('diner', 2)

# for word, cosine_distance in synonyms:
#     print("{}: {}".format(word, cosine_distance))

shitty: 0.7646585702896118
slice,: 0.7196394205093384


In [ ]:
# def cosine_similarity(x, y):
#     return x.dot(y)/(x.norm(2)*y.norm(2))

# x = model.transform('diner')
# y = model.transform('slice,')
# cosine_similarity(x, y)

0.719639502741139